In [1]:
import numpy as np
import os
import cv2
import face_recognition
from datetime import datetime

In [2]:
path = r"C:\Users\Harsh\Pictures\ImageAttendance"
images = []
classnames = []

# Get list of image files from the directory
myList = os.listdir(path)
print(myList)

for cl in myList:
    curImg = cv2.imread(f"{path}/{cl}")  # Read the image
    images.append(curImg)  # Add the image to the list

    # Extract the name before the underscore, e.g., "harsh" from "harsh_1"
    name = os.path.splitext(cl)[0].split('_')[0]
    classnames.append(name)  # Append the base name (without number)

print(classnames)  # Check the list of names


['Harsh_1.jpg', 'Harsh_2.jpg', 'Harsh_3.jpg', 'Harsh_4.jpg', 'Harsh_5.jpg', 'janak_1.jpg', 'janak_2.jpg', 'janak_3.jpg', 'janak_4.jpg', 'janak_5.jpg']
['Harsh', 'Harsh', 'Harsh', 'Harsh', 'Harsh', 'janak', 'janak', 'janak', 'janak', 'janak']


In [3]:
def findEncodings(images):
    encodelist = []  # List to store encodings
    for img in images:
        # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Find face encodings for the current image
        encodes = face_recognition.face_encodings(img)
        
        # If a face encoding was found, append it to the list
        if len(encodes) > 0:
            encodelist.append(encodes[0])
        else:
            print("No face found in image, skipping...")
    
    return encodelist  # Return after all images are processed


In [4]:
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()  # Read all existing lines in the CSV
        nameList = []  # A list to store all the names already recorded

        # Extract names from each line and add them to nameList
        for line in myDataList:
            entry = line.split(',')  # Split line by comma to get the name
            nameList.append(entry[0])  # Append only the name (first item)

        # If the name is not already in the file, write it to the file
        if name not in nameList:
            now = datetime.now()  # Get the current time
            dtString = now.strftime('%H:%M:%S')  # Format the time as HH:MM:SS
            f.write(f'\n{name},{dtString}')  # Write name and time to the file


In [5]:
encodelistknown = findEncodings(images)
print ("Encoding complete")

No face found in image, skipping...
No face found in image, skipping...
No face found in image, skipping...
Encoding complete


In [9]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()  
    if not success:
        break  

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)  # Resize for faster processing
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)  # Convert to RGB as face_recognition uses RGB

    facesCurFrame = face_recognition.face_locations(imgS)  # Detect face locations
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)  # Get face encodings

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodelistknown, encodeFace)
        faceDis = face_recognition.face_distance(encodelistknown, encodeFace)
        matchIndex = np.argmin(faceDis)

        # If a match is found
        if matches[matchIndex]:
            name = classnames[matchIndex].upper()  # Get the name of the matched person
        else:
            name = "Unknown"  # If no match, label as "Unknown"

        y1, x2, y2, x1 = faceLoc  # Unpack face location
        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4  # Scale back to original size

        # Draw rectangle around the face
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)    
        # Draw filled rectangle below the face for the name
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        # Put the name below the face
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Call markAttendance only if the person is recognized (not "Unknown")
        if name != "Unknown":
            markAttendance(name)

    # Show the webcam image
    cv2.imshow('Webcam', img)

    # Break loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close all OpenCV windows
